In [2]:
from rfpipe import pipeline
# Updated for rfpipe version 1.3.1
from rfpipe import candidates
import rfpipe
import numpy as np 
import pylab as plt
import matplotlib
import logging
import glob
logger = logging.getLogger('rfpipe')
%matplotlib inline
from refine_utils import cd_refine

/lustre/aoc/observers/nm-10222/anaconda3/envs/fetchrf/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/lustre/aoc/observers/nm-10222/anaconda3/envs/fetchrf/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
params = {
        'axes.labelsize' : 14,
        'font.size' : 9,
        'legend.fontsize': 12,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'text.usetex': False,
        'figure.figsize': [12, 10]
        }
matplotlib.rcParams.update(params)

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Set up data paths

In [4]:
datadir = '/lustre/aoc/projects/fasttransients/staging/'

SDMmininame = 'realfast_18B-405_TEST_R3_A_cut.58665.73435530093_1562005608240'
miniSDMlist = glob.glob(datadir+SDMmininame)
workdir = '/lustre/aoc/observers/nm-10222/rfgpu/Refinement/nbs/'
gainpath = '/home/mchammer/evladata/telcal/'

In [5]:
gainfile = []
for sdm in miniSDMlist:
    sdmname = sdm.split('/')[-1]
    datasetId = '{0}'.format('_'.join(sdmname.split('_')[1:-1]))
    # # set the paths to the gainfile
    gainname = datasetId + '.GN'
    logging.info('Searching for the gainfile {0} in {1}'.format(gainname, gainpath))
    for path, dirs, files in os.walk(gainpath):
        for f in filter(lambda x: gainname in x, files):
            gainfile.append(os.path.join(path, gainname))
            break

2019-08-15 13:03:58,214 - root - INFO - Searching for the gainfile 18B-405_TEST_R3_A_cut.58665.73435530093.GN in /home/mchammer/evladata/telcal/


# Inputs

In [6]:
preffile = datadir+'realfast.yml'
search_sigma = 7
refine = True
classify = True
fine_dm_search = True

if fine_dm_search:
    dm = 100
    dm_steps = 100
    dm_frac = 0.2

In [7]:
prefs={'saveplots': True, 'savenoise': False, 'savesols': False,
       'savecandcollection': True, 'savecanddata': True,
       'applyonlineflags': True, 'fftmode': 'cuda', 'clustercands': (4,3), 'maxdm':500}

if fine_dm_search:
    logging.info('Setting DM array around DM {0}, from {1} to {2} in {3} steps'.
                 format(dm, dm-dm_frac*dm, dm+dm_frac*dm, dm_steps))
    prefs['dmarr'] = list(np.linspace(dm-dm_frac*dm, dm+dm_frac*dm, dm_steps))

2019-08-15 13:05:48,450 - root - INFO - Setting DM array around DM 100, from 80.0 to 120.0 in 100 steps


# Searching all miniSDMs

In [8]:
for index, sdm in enumerate(miniSDMlist):
    st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdm), sdmscan=1, inprefs = prefs)
    st.prefs.workdir = workdir
    st.prefs.gainfile = gainfile[index]
    st.prefs.sigma_image1 = search_sigma
    
    cc = pipeline.pipeline_scan(st)

2019-08-15 13:05:48,503 - rfpipe.metadata - INFO - Reading metadata from /lustre/aoc/projects/fasttransients/staging/realfast_18B-405_TEST_R3_A_cut.58665.73435530093_1562005608240, scan 1
2019-08-15 13:05:49,483 - rfpipe.state - INFO - Metadata summary:
2019-08-15 13:05:49,483 - rfpipe.state - INFO - 	 Working directory and fileroot: /lustre/aoc/observers/nm-10222/rfgpu/Refinement/nbs, realfast_18B-405_TEST_R3_A_cut.58665.73435530093_1562005608240.1.1
2019-08-15 13:05:49,484 - rfpipe.state - INFO - 	 Using scan 1, source R3
2019-08-15 13:05:49,484 - rfpipe.state - INFO - 	 nants, nbl: 21, 210
2019-08-15 13:05:49,485 - rfpipe.state - INFO - 	 nchan, nspw: 896, 16
2019-08-15 13:05:49,487 - rfpipe.state - INFO - 	 Freq range: 1.012 -- 2.027
2019-08-15 13:05:49,487 - rfpipe.state - INFO - 	 Scan has 431 ints (4.3 s) and inttime 0.010 s
2019-08-15 13:05:49,488 - rfpipe.state - INFO - 	 2 polarizations: ['RR', 'LL']
2019-08-15 13:05:49,489 - rfpipe.state - INFO - 	 Ideal uvgrid npix=(4608, 5

2019-08-15 13:06:34,892 - rfpipe.search - INFO - Imaging 167 ints (0-166) in seg 0 at DM/dt 87.3/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13:06:35,229 - rfpipe.search - INFO - Got one! SNR1 7.1 candidate at (0, 52, 18, 0, 0) and (l, m) = (-0.00169, -0.00487)
2019-08-15 13:06:35,959 - rfpipe.search - INFO - Imaging 167 ints (0-166) in seg 0 at DM/dt 87.7/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13:06:36,296 - rfpipe.search - INFO - Got one! SNR1 7.5 candidate at (0, 52, 19, 0, 0) and (l, m) = (-0.00169, -0.00487)
2019-08-15 13:06:37,026 - rfpipe.search - INFO - Imaging 167 ints (0-166) in seg 0 at DM/dt 88.1/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13:06:37,363 - rfpipe.search - INFO - Got one! SNR1 7.4 candidate at (0, 52, 20, 0, 0) and (l, m) = (-0.00169, -0.00487)
2019-08-15 13:06:38,093 - rfpipe.search - INFO - Imaging 167 ints (0-166) in seg 0 at DM/dt 88.5/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13:06:38,430 - rfpipe.search - IN

2019-08-15 13:07:30,315 - rfpipe.search - INFO - Imaging 160 ints (0-159) in seg 0 at DM/dt 108.7/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13:07:31,337 - rfpipe.search - INFO - Imaging 160 ints (0-159) in seg 0 at DM/dt 109.1/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13:07:32,360 - rfpipe.search - INFO - Imaging 160 ints (0-159) in seg 0 at DM/dt 109.5/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13:07:33,381 - rfpipe.search - INFO - Imaging 160 ints (0-159) in seg 0 at DM/dt 109.9/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13:07:34,403 - rfpipe.search - INFO - Imaging 160 ints (0-159) in seg 0 at DM/dt 110.3/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13:07:35,425 - rfpipe.search - INFO - Imaging 160 ints (0-159) in seg 0 at DM/dt 110.7/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13:07:36,446 - rfpipe.search - INFO - Imaging 160 ints (0-159) in seg 0 at DM/dt 111.1/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-15 13

ZeroDivisionError: division by zero

# Classify and generate refinement plots

In [ ]:
# Classify the generated pickles using FETCH and generate refinement plots
for pkl in glob.glob(st.prefs.workdir+'/'+'cands_*'+sdmname.split('/')[0]+'*.pkl'):
    if classify or refine:
        logging.info('Refining and classifying pkl: {0}'.format(pkl))
        ccs = list(candidates.iter_cands(pkl, select='candcollection'))
        for cc in ccs:
            cds = cc.canddata
            if cds:
                for cd in cds:
                    if classify:
                        payload = candidates.cd_to_fetch(cd, classify=True, save_png=True, show=True, mode = 'GPU')
                        logging.info('FETCH FRB Probability of the candidate {0} is {1}'.format(cd.candid, payload))
                    if refine:
                        logging.info('Generating Refinement plots')
                        cd_refine(cd)
            else:
                logging.info('No candidate was found in cc: {0}'.format(cc))